<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Final Project: Data Analysis using Spark

Estimated time needed: **60** minutes

This final project is similar to the Practice Project you did. In this project, you will not be provided with hints or solutions. You will create a DataFrame by loading data from a CSV file and apply transformations and actions using Spark SQL. This needs to be achieved by performing the following tasks:

- Task 1: Generate DataFrame from CSV data.
- Task 2: Define a schema for the data.
- Task 3: Display schema of DataFrame.
- Task 4: Create a temporary view.
- Task 5: Execute an SQL query.
- Task 6: Calculate Average Salary by Department.
- Task 7: Filter and Display IT Department Employees.
- Task 8: Add 10% Bonus to Salaries.
- Task 9: Find Maximum Salary by Age.
- Task 10: Self-Join on Employee Data.
- Task 11: Calculate Average Employee Age.
- Task 12: Calculate Total Salary by Department.
- Task 13: Sort Data by Age and Salary.
- Task 14: Count Employees in Each Department.
- Task 15: Filter Employees with the letter o in the Name.


### Prerequisites 

1. For this lab assignment, you will be using Python and Spark (PySpark). Therefore, it's essential to make sure that the following libraries are installed in your lab environment or within Skills Network (SN) Labs


In [1]:
# # Installing required packages  

# !pip install pyspark  findspark wget

In [2]:
import findspark
findspark.init()

In [3]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the SparkContext.
from pyspark import SparkConf
from pyspark.sql import SparkSession

In [4]:
config = SparkConf()
config.setAppName('Big Data Final Project') \
      .setAll([
          ("spark.sql.legacy.timeParserPolicy", "LEGACY"),
          ("spark.executor.memory", "12g"),
          ("spark.driver.memory", "1g"),
          ("spark.executor.cores", "3")
    ])

# Creating a SparkSession  
spark = SparkSession.builder.config(conf=config).getOrCreate()

25/12/07 18:12:58 WARN Utils: Your hostname, MICKYXPS15 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/07 18:12:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 18:12:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


2. Download the CSV data.  


In [5]:
# Download the CSV data first into a local `employees.csv` file
import wget, os

if 'employees.csv' not in os.listdir():
    wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/data/employees.csv")

### Tasks


#### Task 1: Generate a Spark DataFrame from the CSV data

Read data from the provided CSV file, `employees.csv` and import it into a Spark DataFrame variable named `employees_df`.

 


In [6]:
# Read data from the "emp" CSV file and import it into a DataFrame variable named "employees_df"  
employees_df = spark.read.csv('employees.csv', header=True, inferSchema=False)

#### Task 2: Define a schema for the data

Construct a schema for the input data and then utilize the defined schema to read the CSV file to create a DataFrame named `employees_df`.  


In [7]:
# Define a Schema for the input data and read the file using the user-defined Schema
from pyspark.sql.functions import udf as spark_udf
from pyspark.sql.types import IntegerType, StringType, DoubleType, StructField, StructType

user_schema = StructType([
    StructField('Emp_No', IntegerType(), True),
    StructField('Emp_Name', StringType(), True),
    StructField('Salary', DoubleType(), True),
    StructField('Age', IntegerType(), True),
    StructField('Department', StringType(), True)
])

employees_df = spark.read.csv('employees.csv', schema=user_schema, header=True)

#### Task 3: Display schema of DataFrame

Display the schema of the `employees_df` DataFrame, showing all columns and their respective data types.  


In [8]:
# Display all columns of the DataFrame, along with their respective data types
employees_df.printSchema()

root
 |-- Emp_No: integer (nullable = true)
 |-- Emp_Name: string (nullable = true)
 |-- Salary: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Department: string (nullable = true)



#### Task 4: Create a temporary view

Create a temporary view named `employees` for the `employees_df` DataFrame, enabling Spark SQL queries on the data. 


In [9]:
# Create a temporary view named "employees" for the DataFrame
employees_df.createTempView('employees')

#### Task 5: Execute an SQL query

Compose and execute an SQL query to fetch the records from the `employees` view where the age of employees exceeds 30. Then, display the result of the SQL query, showcasing the filtered records.


In [10]:
# SQL query to fetch solely the records from the View where the age exceeds 30
spark.sql('SELECT * FROM employees WHERE Age > 30').sort('Age', ascending=True).show()

+------+-----------+-------+---+----------+
|Emp_No|   Emp_Name| Salary|Age|Department|
+------+-----------+-------+---+----------+
|   110|       John| 8200.0| 31| Marketing|
|   127|      James| 2400.0| 31|        HR|
|   201|    Michael|13000.0| 32|        IT|
|   111|     Ismael| 7700.0| 32|        IT|
|   119|      Karen| 2500.0| 32|   Finance|
|   205|    Shelley|12008.0| 33|   Finance|
|   117|      Sigal| 2800.0| 33|     Sales|
|   124|      Kevin| 5800.0| 33| Marketing|
|   128|     Steven| 2200.0| 33|   Finance|
|   199|    Douglas| 2600.0| 34|     Sales|
|   112|Jose Manuel| 7800.0| 34|        HR|
|   113|       Luis| 6900.0| 34|     Sales|
|   132|         TJ| 2100.0| 34|        IT|
|   107|      Diana| 4200.0| 35|     Sales|
|   109|     Daniel| 9000.0| 35|        HR|
|   123|     Shanta| 6500.0| 35|     Sales|
|   125|      Julia| 3200.0| 35|   Finance|
|   135|         Ki| 2400.0| 35| Marketing|
|   138|    Stephen| 3200.0| 35|     Sales|
|   200|   Jennifer| 4400.0| 36|

#### Task 6: Calculate Average Salary by Department

Compose an SQL query to retrieve the average salary of employees grouped by department. Display the result.


In [11]:
# SQL query to calculate the average salary of employees grouped by department
query = '''SELECT Department, ROUND(AVG(Salary), 2) AS `Average Salary` FROM employees 
           GROUP BY Department
        '''
spark.sql(query).show()

+----------+--------------+
|Department|Average Salary|
+----------+--------------+
|     Sales|       5492.92|
|        HR|        5837.5|
|   Finance|        5730.8|
| Marketing|       6633.33|
|        IT|        7400.0|
+----------+--------------+



#### Task 7: Filter and Display IT Department Employees

Apply a filter on the `employees_df` DataFrame to select records where the department is `'IT'`. Display the filtered DataFrame.


In [12]:
# Apply a filter to select records where the department is 'IT'
employees_df.filter('Department == "IT"').show()

+------+--------+-------+---+----------+
|Emp_No|Emp_Name| Salary|Age|Department|
+------+--------+-------+---+----------+
|   198|  Donald| 2600.0| 29|        IT|
|   201| Michael|13000.0| 32|        IT|
|   206| William| 8300.0| 37|        IT|
|   100|  Steven|24000.0| 39|        IT|
|   104|   Bruce| 6000.0| 38|        IT|
|   105|   David| 4800.0| 39|        IT|
|   111|  Ismael| 7700.0| 32|        IT|
|   129|   Laura| 3300.0| 38|        IT|
|   132|      TJ| 2100.0| 34|        IT|
|   136|   Hazel| 2200.0| 29|        IT|
+------+--------+-------+---+----------+



#### Task 8: Add 10% Bonus to Salaries

Perform a transformation to add a new column named "SalaryAfterBonus" to the DataFrame. Calculate the new salary by adding a 10% bonus to each employee's salary.


In [13]:
from pyspark.sql.functions import col as spark_col, try_add as spark_add, try_multiply as spark_multiply, lit

# Add a new column "SalaryAfterBonus" with 10% bonus added to the original salary
employees_df = employees_df.withColumn('SalaryAfterBonus', spark_add(spark_col('Salary'), spark_multiply(spark_col('Salary'), lit(0.1))))

#### Task 9: Find Maximum Salary by Age

Group the data by age and calculate the maximum salary for each age group. Display the result.


In [14]:
from pyspark.sql.functions import max as spark_max

# Group data by age and calculate the maximum salary for each age group
employees_df.groupBy('Age').agg(spark_max('Salary')).show()

+---+-----------+
|Age|max(Salary)|
+---+-----------+
| 31|     8200.0|
| 34|     7800.0|
| 28|    12008.0|
| 27|    17000.0|
| 26|     3600.0|
| 37|    17000.0|
| 35|     9000.0|
| 39|    24000.0|
| 38|     6000.0|
| 29|    10000.0|
| 32|    13000.0|
| 33|    12008.0|
| 30|     8000.0|
| 36|     7900.0|
+---+-----------+



#### Task 10: Self-Join on Employee Data

Join the "employees_df" DataFrame with itself based on the "Emp_No" column. Display the result.


In [15]:
# Join the DataFrame with itself based on the "Emp_No" column
employees_df.join(employees_df, 'Emp_No', 'inner').show()

+------+---------+-------+---+----------+----------------+---------+-------+---+----------+----------------+
|Emp_No| Emp_Name| Salary|Age|Department|SalaryAfterBonus| Emp_Name| Salary|Age|Department|SalaryAfterBonus|
+------+---------+-------+---+----------+----------------+---------+-------+---+----------+----------------+
|   198|   Donald| 2600.0| 29|        IT|          2860.0|   Donald| 2600.0| 29|        IT|          2860.0|
|   199|  Douglas| 2600.0| 34|     Sales|          2860.0|  Douglas| 2600.0| 34|     Sales|          2860.0|
|   200| Jennifer| 4400.0| 36| Marketing|          4840.0| Jennifer| 4400.0| 36| Marketing|          4840.0|
|   201|  Michael|13000.0| 32|        IT|         14300.0|  Michael|13000.0| 32|        IT|         14300.0|
|   202|      Pat| 6000.0| 39|        HR|          6600.0|      Pat| 6000.0| 39|        HR|          6600.0|
|   203|    Susan| 6500.0| 36| Marketing|          7150.0|    Susan| 6500.0| 36| Marketing|          7150.0|
|   204|  Hermann|1

#### Task 11: Calculate Average Employee Age

Calculate the average age of employees using the built-in aggregation function. Display the result.


In [ ]:
# Calculate the average age of employees
from pyspark.sql.functions import avg as spark_avg

employees_df.agg(spark_avg('Age')).show()

+--------+
|avg(Age)|
+--------+
|   33.56|
+--------+



#### Task 12: Calculate Total Salary by Department

Calculate the total salary for each department using the built-in aggregation function. Display the result.


In [20]:
# Calculate the total salary for each department. Hint - User GroupBy and Aggregate functions
from pyspark.sql.functions import sum as spark_sum

employees_df.groupBy('Department').agg(spark_sum('Salary')).show()

+----------+-----------+
|Department|sum(Salary)|
+----------+-----------+
|     Sales|    71408.0|
|        HR|    46700.0|
|   Finance|    57308.0|
| Marketing|    59700.0|
|        IT|    74000.0|
+----------+-----------+



#### Task 14: Count Employees in Each Department

Calculate the number of employees in each department. Display the result.


#### Task 13: Sort Data by Age and Salary

Apply a transformation to sort the DataFrame by age in ascending order and then by salary in descending order. Display the sorted DataFrame.


In [18]:
# Sort the DataFrame by age in ascending order and then by salary in descending order
employees_df.sort(['Age', 'Salary'], ascending=[True, False]).show()

+------+---------+-------+---+----------+----------------+
|Emp_No| Emp_Name| Salary|Age|Department|SalaryAfterBonus|
+------+---------+-------+---+----------+----------------+
|   137|   Renske| 3600.0| 26| Marketing|          3960.0|
|   101|    Neena|17000.0| 27|     Sales|         18700.0|
|   114|      Den|11000.0| 27|   Finance|         12100.0|
|   108|    Nancy|12008.0| 28|     Sales|         13208.8|
|   130|    Mozhe| 2800.0| 28| Marketing|          3080.0|
|   126|    Irene| 2700.0| 28|        HR|          2970.0|
|   204|  Hermann|10000.0| 29|   Finance|         11000.0|
|   115|Alexander| 3100.0| 29|   Finance|          3410.0|
|   134|  Michael| 2900.0| 29|     Sales|          3190.0|
|   198|   Donald| 2600.0| 29|        IT|          2860.0|
|   140|   Joshua| 2500.0| 29|   Finance|          2750.0|
|   136|    Hazel| 2200.0| 29|        IT|          2420.0|
|   120|  Matthew| 8000.0| 30|        HR|          8800.0|
|   110|     John| 8200.0| 31| Marketing|          9020.

In [ ]:
from pyspark.sql.functions import count as spark_count

# Calculate the number of employees in each department
employees_df.groupBy('Department').agg(spark_count('Emp_No')).show()

+----------+-------------+
|Department|count(Emp_No)|
+----------+-------------+
|     Sales|           13|
|        HR|            8|
|   Finance|           10|
| Marketing|            9|
|        IT|           10|
+----------+-------------+



#### Task 15: Filter Employees with the letter o in the Name

Apply a filter to select records where the employee's name contains the letter `'o'`. Display the filtered DataFrame.


In [21]:
# Apply a filter to select records where the employee's name contains the letter 'o'
employees_df.filter(employees_df.Emp_Name.contains('o')).show()

+------+-----------+------+---+----------+----------------+
|Emp_No|   Emp_Name|Salary|Age|Department|SalaryAfterBonus|
+------+-----------+------+---+----------+----------------+
|   198|     Donald|2600.0| 29|        IT|          2860.0|
|   199|    Douglas|2600.0| 34|     Sales|          2860.0|
|   110|       John|8200.0| 31| Marketing|          9020.0|
|   112|Jose Manuel|7800.0| 34|        HR|          8580.0|
|   130|      Mozhe|2800.0| 28| Marketing|          3080.0|
|   133|      Jason|3300.0| 38|     Sales|          3630.0|
|   139|       John|2700.0| 36|     Sales|          2970.0|
|   140|     Joshua|2500.0| 29|   Finance|          2750.0|
+------+-----------+------+---+----------+----------------+



In [ ]:
spark.stop()

# Congratulations! You have completed the project.

Now you know how to create a DataFrame from a CSV data file and perform a variety of DataFrame transformations and actions using Spark SQL.


## Authors


Raghul Ramesh


Lavanya T S


<!--## Change Log -->


<!--|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-09-01|0.1|Lavanya T S|Initial version|
|2023-09-11|0.2|Pornima More|QA pass with edits|-->


<h3 align="center"> &#169; IBM Corporation. All rights reserved. <h3/>
